In [71]:
import os
import cv2
import json
import matplotlib.pyplot as plt
from PIL import Image
from sentence_transformers import SentenceTransformer, util
from io import BytesIO

In [2]:
model = SentenceTransformer('clip-ViT-B-32')

/home/engine210/MMFinal/torch-1.7/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [4]:
def read_json_data(file_name):
    with open(file_name) as f:
        article_list = [json.loads(line) for line in f]
        return article_list

In [6]:
BASE_DIR = '/home/engine210/MMFinal/'
DATA_DIR = '/home/engine210/MMFinal/data/'
TARGET_DIR = "/home/engine210/MMFinal/viz/"

In [135]:
test_samples = read_json_data(os.path.join(DATA_DIR, 'mmsys_anns', 'public_test_mmsys_final.json'))
v_data = test_samples[445]

In [136]:
def get_scores(v_data):
    img_path = os.path.join(DATA_DIR, v_data["img_local_path"])
    img = Image.open(img_path)
    img_shape = img.size

    bbox_list = v_data['maskrcnn_bboxes']
    bbox_list.append([0, 0, img_shape[0], img_shape[1]])  # For entire image (global context)

    cap1 = v_data['caption1_modified']
    cap2 = v_data['caption2_modified']
    # cap1 = v_data['caption1']
    # cap2 = v_data['caption2']

    print(cap1)
    print(cap2)
    print(img.size)
    score_c1 = []
    score_c2 = []

    for i, box in enumerate(bbox_list):
        box = list(map(int, box))
        if box[0] == box[2]: box[2] += 1
        if box[1] == box[3]: box[3] += 1
        crop_img = img.crop(box)
        b = BytesIO()
        crop_img.save(b,format="jpeg")
        crop_img = Image.open(b)
        # crop_img.save('tmp'+str(i)+'.jpg')
        crop_img_emb = model.encode(crop_img)
        text_emb = model.encode([cap1, cap2])

        # #Compute cosine similarities 
        cos_scores = util.cos_sim(crop_img_emb, text_emb)
        # print(i, cos_scores.numpy()[0])
        score_c1.append(cos_scores.numpy()[0][0])
        score_c2.append(cos_scores.numpy()[0][1])
    return score_c1, score_c2

In [137]:
score_c1, score_c2 = get_scores(v_data)
print(score_c1.index(max(score_c1)), score_c2.index(max(score_c2)))

President PERSON's ORDINAL wife, PERSON, died in a plane crash in DATE.
There is no truth to the sensational headline that PERSON’s Ex-Wife PERSON Died After Airline Accident
(865, 452)
7 0


In [139]:
v_data['context_label'] == 1

True

In [14]:
def draw_bboxes(data):
    box_color = (0, 255, 0)  # Green
    line_thickness = 5
    img_path = os.path.join(DATA_DIR, v_data['img_local_path'])
    img = cv2.imread(img_path)
    bboxes = data["maskrcnn_bboxes"]

    for i, bbox in enumerate(bboxes):
        img = cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), box_color, line_thickness)
    cv2.imwrite('tmp.jpg', img)

In [15]:
draw_bboxes(v_data)

In [16]:
img = Image.open(os.path.join(DATA_DIR, v_data['img_local_path']))

In [18]:
img1 = img.crop(v_data['maskrcnn_bboxes'][0])

In [19]:
img1.save('tmp1.jpg')

In [17]:
v_data['maskrcnn_bboxes']

[[280.65081787109375,
  24.754322052001953,
  525.8712768554688,
  396.9060974121094],
 [61.683773040771484,
  68.53385162353516,
  251.91490173339844,
  397.5182189941406],
 [386.9053955078125, 311.1746520996094, 422.8839416503906, 399.09246826171875],
 [562.51953125, 123.96571350097656, 600.0, 275.7099304199219],
 [230.67855834960938, 201.62765502929688, 287.44677734375, 389.9917907714844],
 [385.1832275390625, 311.67584228515625, 422.2655334472656, 383.9637756347656],
 [225.81739807128906,
  184.658935546875,
  252.52146911621094,
  237.91329956054688],
 [384.7476806640625, 165.92344665527344, 505.6837158203125, 396.7842712402344],
 [216.56350708007812,
  170.42654418945312,
  262.8428039550781,
  237.44952392578125],
 [259.2336120605469, 230.63587951660156, 286.9748229980469, 258.78125]]

error: OpenCV(4.5.2) /tmp/pip-req-build-13uokl4r/opencv/modules/highgui/src/window.cpp:404: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [6]:
bboxes = v_data['maskrcnn_bboxes']
top_bbox_c1 = top_bbox_from_scores(bboxes, score_c1)
top_bbox_c2 = top_bbox_from_scores(bboxes, score_c2)
print(top_bbox_c1, top_bbox_c2)

NameError: name 'score_c1' is not defined

In [15]:
score_c1, score_c2 = get_scores(v_data)

In [16]:
print(score_c1, score_c2)

tensor([-1.3500,  0.1246, -3.7564,  5.0216, -1.3501, -1.4075, -1.2110, -1.2906,
        -1.3245, -2.4942,  2.6325], device='cuda:0') tensor([-3.2680,  1.2437, -3.6021,  4.2952, -1.1264,  3.6721, -1.2736, -0.7548,
        -3.2077, -1.1968,  6.7783], device='cuda:0')


In [12]:
print([list(map(int, x)) for x in v_data['maskrcnn_bboxes']])

[[389, 72, 505, 373], [89, 312, 190, 396], [116, 225, 161, 288], [180, 225, 207, 271], [579, 193, 597, 249], [217, 225, 256, 267], [67, 237, 92, 275], [29, 240, 64, 276], [229, 298, 251, 330], [89, 205, 104, 228], [0, 0, 600, 400]]


In [6]:
def get_scores(v_data):
    """
        Computes score for the two captions associated with the image

        Args:
            v_data (dict): A dictionary holding metadata about on one data sample

        Returns:
            score_c1 (float): Score for the first caption associated with the image
            score_c2 (float): Score for the second caption associated with the image
    """
    checkpoint = torch.load(BASE_DIR + 'models/' + model_name + '.pt')
    combined_model.load_state_dict(checkpoint)
    combined_model.to(device)
    combined_model.eval()

    img_path = os.path.join(DATA_DIR, v_data["img_local_path"])
    bbox_list = v_data['maskrcnn_bboxes']
    bbox_classes = [-1] * len(bbox_list)
    img = cv2.imread(img_path)
    img_shape = img.shape[:2]
    bbox_list.append([0, 0, img_shape[1], img_shape[0]])  # For entire image (global context)
    bbox_classes.append(-1)
    cap1 = v_data['caption1_modified']
    cap2 = v_data['caption2_modified']

    img_tensor = [torch.tensor(img).to(device)]
    bboxes = [torch.tensor(bbox_list).to(device)]
    bbox_classes = [torch.tensor(bbox_classes).to(device)]

    if embed_type != 'use':
        # For Glove, Fasttext embeddings
        cap1_p = text_field.preprocess(cap1)
        cap2_p = text_field.preprocess(cap2)
        embed_c1 = torch.stack([text_field.vocab.vectors[text_field.vocab.stoi[x]] for x in cap1_p]).unsqueeze(
            0).to(device)
        embed_c2 = torch.stack([text_field.vocab.vectors[text_field.vocab.stoi[x]] for x in cap2_p]).unsqueeze(
            0).to(device)
    else:
        # For USE embeddings
        embed_c1 = torch.tensor(use_embed([cap1]).numpy()).to(device)
        embed_c2 = torch.tensor(use_embed([cap2]).numpy()).to(device)

    with torch.no_grad():
        z_img, z_t_c1, z_t_c2 = combined_model(img_tensor, embed_c1, embed_c2, 1, [embed_c1.shape[1]],
                                               [embed_c2.shape[1]], bboxes, bbox_classes)

    z_img = z_img.permute(1, 0, 2)
    z_text_c1 = z_t_c1.unsqueeze(2)
    z_text_c2 = z_t_c2.unsqueeze(2)

    # Compute Scores
    score_c1 = torch.bmm(z_img, z_text_c1).squeeze()
    score_c2 = torch.bmm(z_img, z_text_c2).squeeze()

    return score_c1, score_c2

In [ ]:
ours_correct = 0
lang_correct = 0

for i, v_data in enumerate(test_samples):
    actual_context = int(v_data['context_label'])
    
    language_context = 0 if float(v_data['bert_base_score']) >= textual_sim_threshold else 1

    pred_context = evaluate_context_with_bbox_overlap(v_data)

    if pred_context == actual_context:
        ours_correct += 1

    if language_context == actual_context:
        lang_correct += 1

print("Cosmos Accuracy", ours_correct / len(test_samples))
print("Language Baseline Accuracy", lang_correct / len(test_samples))